# 1. 환경준비

In [1]:
from matplotlib import pyplot as plt
from datetime import datetime as dt

import os
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import random

# Pandas 데이터 프레임에서 float을 소수점 2자리 까지 출력해 준다.
pd.set_option('display.float_format', lambda x: '%.4f' % x)
# Pandas 데이터 프레임에서 column을 다 출력하도록 설정한다.
pd.set_option('display.max_columns', None)
# Pandas 데이터 프레임에서 row를 다 출력하도록 설정한다.
# pd.set_option('display.max_rows', None)

random.seed(3)

In [2]:
data = pd.read_csv('klpga_train.csv', encoding = 'cp949')

# 2. 전처리

## 2-1) 사전논의 컬럼 제거

In [3]:
## 사전 논의된 제거 컬럼
data.drop(['rank_str','event_yard', 'birdie', 'pBreak', 'days', 'prize_cumsum_1', 'year',
           'max_prize','min_prize', 'degree','rank', 'DEA_vrs', 'closeness', 'betweenness',
           'eigenvector', 'birth', 'prize', 'sixty', 'avgBirdie', 'avgPut', 
           'prize_cumsum_1_hhi', 'prize_cumsum_1_gini','drivGIR', 'driving', 
           'avgStrokePerYard', 'iron', 'p4GIR', 'point', 'prize', 'p3GIR', 'all', 'GIR', 
           'fieldPut', 'p3Score', 'p5Score', 'p4Score', 'event_par', 'DEA_crs'], axis = 1, inplace = True)

## 2-2) 날짜 변수 날짜화 & 날짜 인덱스화

In [4]:
data['date'] = pd.to_datetime(data['date'], format = '%Y-%m-%d')
data['month'] = data['date'].dt.month
data.index = data['date']
data.drop('date', axis = 1, inplace = True)

## 2-3) 인코딩 I

In [5]:
#data['name'] = data['name'].astype(str)
data['event_name'] = data['event_name'].astype(str) ## 결과확인용, 회귀분석에는 안 씀
data['event_code'] = data['event_code'].astype(str)
data['series_code'] = data['series_code'].astype(str)
data['sponsor'] = data['sponsor'].astype(str)
data['month'] = data['month'].astype(str)

In [6]:
#data = pd.get_dummies(data, columns = ['name'], prefix = 'Name')
data = pd.get_dummies(data, columns = ['event_code'], prefix = 'Event_code')
data = pd.get_dummies(data, columns = ['series_code'], prefix = 'Series_code')
data = pd.get_dummies(data, columns = ['sponsor'], prefix = 'Sponsor')
data = pd.get_dummies(data, columns = ['month'], prefix = 'Month')

# 2-4) 독립변수 (t-1)(t-2)(t-3)/3

In [7]:
data.head()

,name,event_name,event_meter,rookieP,avgStroke,holeInOne,eagle,p3avgBirdie,p4avgBirdie,p5avgBirdie,drivAcc,driveDist,p3Put,p4Put,p5Put,p5GIR,bunkSave,recovery,top10,hit,avgHoleInOne,avgEagle,round,height,age,event_cumsum,prize_cumsum,prize_gini,prize_hhi,career,top1,top2,total_prize,prize_cumsum_gini,prize_cumsum_hhi,Event_code_2008040003,Event_code_2008040004,Event_code_2008080009,Event_code_2008090006,Event_code_2008090009,Event_code_2008100005,Event_code_2008100007,Event_code_2008110002,Event_code_2008110003,Event_code_2009040002,Event_code_2009050002,Event_code_2009080003,Event_code_2010040001,Event_code_2010040004,Event_code_2010050005,Event_code_2010050011,Event_code_2010060004,Event_code_2010060005,Event_code_2010070005,Event_code_2010080003,Event_code_2010080004,Event_code_2010090002,Event_code_2010090003,Event_code_2010100003,Event_code_2010110003,Event_code_2010110004,Event_code_2011040002,Event_code_2011040003,Event_code_2011050008,Event_code_2011060003,Event_code_2011060006,Event_code_2011060012,Event_code_2011070005,Event_code_2011080008,Event_code_2011080011,Event_code_2011090009,Event_code_2011090012,Event_code_2011100006,Event_code_2011100009,Event_code_2011110003,Event_code_2011110007,Event_code_2012040002,Event_code_2012040003,Event_code_2012050009,Event_code_2012060016,Event_code_2012060017,Event_code_2012090013,Event_code_2012090014,Event_code_2012100010,Event_code_2012100011,Event_code_2012110004,Event_code_2013040002,Event_code_2013040007,Event_code_2013050005,Event_code_2013050006,Event_code_2013050008,Event_code_2013060010,Event_code_2013060011,Event_code_2013080011,Event_code_2013080012,Event_code_2013090008,Event_code_2013090009,Event_code_2013090010,Event_code_2013100012,Event_code_2013100013,Event_code_2013110006,Event_code_2013110007,Event_code_2014040003,Event_code_2014040005,Event_code_2014050003,Event_code_2014050005,Event_code_2014050007,Event_code_2014050009,Event_code_2014060001,Event_code_2014060003,Event_code_2014060007,Event_code_2014070002,Event_code_2014070021,Event_code_2014080001,Event_code_2014080002,Event_code_2014080003,Event_code_2014090001,Event_code_2014090002,Event_code_2014100001,Event_code_2014100002,Event_code_2014100004,Event_code_2014100005,Event_code_2014110001,Event_code_2014110002,Event_code_2015040003,Event_code_2015040009,Event_code_2015040010,Event_code_2015050003,Event_code_2015050005,Event_code_2015050007,Event_code_2015050014,Event_code_2015060006,Event_code_2015060007,Event_code_2015060018,Event_code_2015070006,Event_code_2015070007,Event_code_2015070008,Event_code_2015080007,Event_code_2015080008,Event_code_2015080017,Event_code_2015090005,Event_code_2015090006,Event_code_2015090007,Event_code_2015090008,Event_code_2015100011,Event_code_2015100012,Event_code_2015100015,Event_code_2015110002,Event_code_2015110003,Event_code_2016040002,Event_code_2016040003,Event_code_2016040004,Event_code_2016040005,Event_code_2016050001,Event_code_2016050002,Event_code_2016050004,Event_code_2016060001,Event_code_2016060002,Event_code_2016060004,Event_code_2016070002,Event_code_2016070003,Event_code_2016070004,Event_code_2016070005,Event_code_2016080001,Event_code_2016080002,Event_code_2016080003,Event_code_2016090001,Event_code_2016090002,Event_code_2016090003,Event_code_2016090004,Event_code_2016100001,Event_code_2016100004,Event_code_2016110001,Event_code_2016110002,Event_code_2017040002,Event_code_2017040003,Event_code_2017040004,Event_code_2017050001,Event_code_2017050002,Event_code_2017050004,Event_code_2017060001,Event_code_2017060002,Event_code_2017060004,Event_code_2017060005,Event_code_2017070002,Event_code_2017070003,Event_code_2017080001,Event_code_2017080002,Event_code_2017080003,Event_code_2017080004,Event_code_2017090001,Event_code_2017090002,Event_code_2017090003,Event_code_2017090004,Event_code_2017100001,Event_code_2017100002,Event_code_2017110001,Event_code_2017110002,Event_code_2018040005,Event_code_2018040006,Event_code_2018040007,Event_code_2018050001,Event_c

In [8]:
data = data.sort_values(['name', 'date'])

In [9]:
data.rename(columns = {'avgStroke' : 'Exp_avgStroke'}, inplace = True) # 평균 낼 과거용

tdata = data.copy() # 종속변수 없는 다 평균 낼 df

data['avgStroke'] = data['Exp_avgStroke'] # 건드리면 안되는 진짜 종속변수!!!

In [10]:
collist = list()
for col in tdata.columns:
    collist.append(col)
collist.remove('name')
collist.remove('event_name')

In [11]:
Groupdata = pd.DataFrame()
Groupdata['name'] = tdata['name']
Groupdata['event_name'] = tdata['event_name']

In [12]:
'avgStroke' in tdata.columns # 확인용

False

In [13]:
'avgStroke' in data.columns

True

In [14]:
for col in collist:
    Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()

C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usu

C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usu

C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usu

C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usu

C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usu

C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata[col] = tdata.groupby('name')[col].shift(1).rolling(window=3).mean()
C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\4001972315.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usu

In [15]:
Groupdata['avgStroke'] = data['avgStroke']
data = Groupdata

C:\Users\H&CONSULTING\AppData\Local\Temp\ipykernel_18344\2847275769.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  Groupdata['avgStroke'] = data['avgStroke']


In [16]:
data.dropna(axis = 0, inplace = True)

In [17]:
data = data.sort_index(ascending = True)

## 2-5) 인코딩 II

In [18]:
data['name'] = data['name'].astype(str)
data = pd.get_dummies(data, columns = ['name'], prefix = 'Name')

# 3. 모델링

## 3-1) Train/Test Set 준비

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = data.copy()
y = X.pop('avgStroke')

In [20]:
scaled_data = scaler.fit_transform(data.select_dtypes(exclude=["object", "category"]))
df_scaled = pd.DataFrame(data=scaled_data, columns=data.select_dtypes(exclude=['object', 'category']).columns)
X = df_scaled.copy()

In [21]:
len(X)

23631

In [22]:
### 데이터 검색 페이지
data.iloc[22600:22601,:]

,event_name,event_meter,rookieP,Exp_avgStroke,holeInOne,eagle,p3avgBirdie,p4avgBirdie,p5avgBirdie,drivAcc,driveDist,p3Put,p4Put,p5Put,p5GIR,bunkSave,recovery,top10,hit,avgHoleInOne,avgEagle,round,height,age,event_cumsum,prize_cumsum,prize_gini,prize_hhi,career,top1,top2,total_prize,prize_cumsum_gini,prize_cumsum_hhi,Event_code_2008040003,Event_code_2008040004,Event_code_2008080009,Event_code_2008090006,Event_code_2008090009,Event_code_2008100005,Event_code_2008100007,Event_code_2008110002,Event_code_2008110003,Event_code_2009040002,Event_code_2009050002,Event_code_2009080003,Event_code_2010040001,Event_code_2010040004,Event_code_2010050005,Event_code_2010050011,Event_code_2010060004,Event_code_2010060005,Event_code_2010070005,Event_code_2010080003,Event_code_2010080004,Event_code_2010090002,Event_code_2010090003,Event_code_2010100003,Event_code_2010110003,Event_code_2010110004,Event_code_2011040002,Event_code_2011040003,Event_code_2011050008,Event_code_2011060003,Event_code_2011060006,Event_code_2011060012,Event_code_2011070005,Event_code_2011080008,Event_code_2011080011,Event_code_2011090009,Event_code_2011090012,Event_code_2011100006,Event_code_2011100009,Event_code_2011110003,Event_code_2011110007,Event_code_2012040002,Event_code_2012040003,Event_code_2012050009,Event_code_2012060016,Event_code_2012060017,Event_code_2012090013,Event_code_2012090014,Event_code_2012100010,Event_code_2012100011,Event_code_2012110004,Event_code_2013040002,Event_code_2013040007,Event_code_2013050005,Event_code_2013050006,Event_code_2013050008,Event_code_2013060010,Event_code_2013060011,Event_code_2013080011,Event_code_2013080012,Event_code_2013090008,Event_code_2013090009,Event_code_2013090010,Event_code_2013100012,Event_code_2013100013,Event_code_2013110006,Event_code_2013110007,Event_code_2014040003,Event_code_2014040005,Event_code_2014050003,Event_code_2014050005,Event_code_2014050007,Event_code_2014050009,Event_code_2014060001,Event_code_2014060003,Event_code_2014060007,Event_code_2014070002,Event_code_2014070021,Event_code_2014080001,Event_code_2014080002,Event_code_2014080003,Event_code_2014090001,Event_code_2014090002,Event_code_2014100001,Event_code_2014100002,Event_code_2014100004,Event_code_2014100005,Event_code_2014110001,Event_code_2014110002,Event_code_2015040003,Event_code_2015040009,Event_code_2015040010,Event_code_2015050003,Event_code_2015050005,Event_code_2015050007,Event_code_2015050014,Event_code_2015060006,Event_code_2015060007,Event_code_2015060018,Event_code_2015070006,Event_code_2015070007,Event_code_2015070008,Event_code_2015080007,Event_code_2015080008,Event_code_2015080017,Event_code_2015090005,Event_code_2015090006,Event_code_2015090007,Event_code_2015090008,Event_code_2015100011,Event_code_2015100012,Event_code_2015100015,Event_code_2015110002,Event_code_2015110003,Event_code_2016040002,Event_code_2016040003,Event_code_2016040004,Event_code_2016040005,Event_code_2016050001,Event_code_2016050002,Event_code_2016050004,Event_code_2016060001,Event_code_2016060002,Event_code_2016060004,Event_code_2016070002,Event_code_2016070003,Event_code_2016070004,Event_code_2016070005,Event_code_2016080001,Event_code_2016080002,Event_code_2016080003,Event_code_2016090001,Event_code_2016090002,Event_code_2016090003,Event_code_2016090004,Event_code_2016100001,Event_code_2016100004,Event_code_2016110001,Event_code_2016110002,Event_code_2017040002,Event_code_2017040003,Event_code_2017040004,Event_code_2017050001,Event_code_2017050002,Event_code_2017050004,Event_code_2017060001,Event_code_2017060002,Event_code_2017060004,Event_code_2017060005,Event_code_2017070002,Event_code_2017070003,Event_code_2017080001,Event_code_2017080002,Event_code_2017080003,Event_code_2017080004,Event_code_2017090001,Event_code_2017090002,Event_code_2017090003,Event_code_2017090004,Event_code_2017100001,Event_code_2017100002,Event_code_2017110001,Event_code_2017110002,Event_code_2018040005,Event_code_2018040006,Event_code_2018040007,Event_code_2018050001,Event_co

In [23]:
X_train = X.iloc[:22600, :]
y_train = y[:22600]
X_test = X.iloc[22600:, :]
y_test = y[22600:]

## 3-2) 성능평가

In [24]:
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor,RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor

!pip install xgboost
from xgboost import XGBRegressor
!pip install catboost
from catboost import CatBoostRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

In [25]:
def scoreCheck(model):
    score = model.score(X_train, y_train)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    
    print('Train Score: ',score)
    print('R^2 :', r2)
    print('Mean Squared Error:', mse)
    print('Mean Absolute Error:', mae)
    print('MAPE:', mape)
    print('RMSE: ', np.sqrt(mse))

In [103]:
print('부스팅 계열 -----------------------------------------------')
'''
# Gradient Boosting
model = GradientBoostingRegressor(n_estimators = 2000, learning_rate = 0.05, max_depth = 5, min_samples_leaf = 15,
                               min_samples_split = 10)
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('Gradient Boosting')
scoreCheck(model)
print('-----------------------------------------------------------')
'''
# XGB
model = XGBRegressor(n_jobs=5, learning_rate=0.1, max_depth=10)
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('XGB')
scoreCheck(model)
print('-----------------------------------------------------------')

# LGBM
model = LGBMRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Light GBM')
scoreCheck(model)
print('-----------------------------------------------------------')

# AdaBoost
model = AdaBoostRegressor()
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('AdaBoost')
scoreCheck(model)
print('-----------------------------------------------------------')

# CatBoost
model = CatBoostRegressor(logging_level ='Silent',iterations=500)
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('CatBoost')
scoreCheck(model)
print('-----------------------------------------------------------')

print('선형 계열 -------------------------------------------------')

# Linear
model = LinearRegression()
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('Linear')
scoreCheck(model)
print('-----------------------------------------------------------')

# ElasticNet
model = ElasticNet(alpha=1.0, l1_ratio=0.5)
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('Elastic Net')
scoreCheck(model)
print('-----------------------------------------------------------')

print('경사하강 계열 ---------------------------------------------')

# SGD 
model = SGDRegressor()
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('SGD')
scoreCheck(model)
print('-----------------------------------------------------------')

print('비모수분류 계열 -------------------------------------------')

# KNN
model = KNeighborsRegressor(n_neighbors = 50)
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('KNN')
scoreCheck(model)
print('-----------------------------------------------------------')

print('트리 계열 -------------------------------------------------')

# RandomForest 
model = RandomForestRegressor()
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('RandomForest')
scoreCheck(model)
print('-----------------------------------------------------------')

# ExtraTrees
model = ExtraTreesRegressor()
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('ExtraTrees')
scoreCheck(model)
print('-----------------------------------------------------------')

부스팅 계열 -----------------------------------------------------------
XGB
Train Score:  0.6651259202811506
R^2 : 0.22343645387172528
Mean Squared Error: 4.627762618657475
Mean Absolute Error: 1.6841313770549489
MAPE: 0.022732555283731855
RMSE:  2.1512235166661493
-----------------------------------------------------------
Light GBM
Train Score:  0.4958699793066572
R^2 : 0.23344230876544947
Mean Squared Error: 4.568134888929818
Mean Absolute Error: 1.6457235216268964
MAPE: 0.022163908064675972
RMSE:  2.137319557045651
-----------------------------------------------------------
AdaBoost
Train Score:  0.14712426892852415
R^2 : 0.13903066826100996
Mean Squared Error: 5.13076065583699
Mean Absolute Error: 1.8404841871294468
MAPE: 0.0251725368762715
RMSE:  2.2651182432352157
-----------------------------------------------------------
CatBoost
Train Score:  0.5404766618172898
R^2 : 0.24546046487432394
Mean Squared Error: 4.4965152850182895
Mean Absolute Error: 1.6458601154727368
MAPE: 0.02221572

# 4. 결과해석

R^2 : Cat / LGBM / XGB = ET / RF
MSE : Cat / LGBM / XGB / ET / RF
MAE : Cat / LGBM / ET / XGB / RF
MAPE: Cat / LGBM / ET / XGB / RF
RMSE: Cat / LGBM / XGB / ET / RF

# 5. 하이퍼파라미터 조정

## 5-1) 준 디폴트

In [32]:
# CatBoost
model = CatBoostRegressor(logging_level ='Silent',iterations=500)
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('CatBoost Default')
scoreCheck(model)

CatBoost Default
Train Score:  0.5396224848104636
R^2 : 0.3301034249440282
Mean Squared Error: 3.992103857909668
Mean Absolute Error: 1.5632588652259556
MAPE: 0.02118577937100895
RMSE:  1.9980249893106112


## 5-2) 조정

In [36]:
# CatBoost
model = CatBoostRegressor(logging_level ='Silent',iterations=750, learning_rate = 0.12, depth = 10)
model.fit(X_train, y_train)
model.predict(X_test)
y_pred = model.predict(X_test)
print('CatBoost Modified')
scoreCheck(model)

CatBoost Modified
Train Score:  0.8457959035944012
R^2 : 0.3493836897053726
Mean Squared Error: 3.877207287004149
Mean Absolute Error: 1.530337783580173
MAPE: 0.020767715502397543
RMSE:  1.9690625401454747


# 6. 예측용

In [ ]:
'''
* 1) 롯데렌터카 여자오픈 04.06-09
22600:22697

* 2) 메디힐/한국일보 챔피언십 04.13-16 
22697:22798

* 3) 넥센/세인트나인 마스터즈 04.21-23
22798:22905

* 4) 크리스에프엔씨 제 45회 KLPGA 챔피언십  04.27-30
22905:23028

* 5) 제 9회 교촌 1991 레이디스 오픈 05.05-07
23028:23141

* 6) 2023 NH 투자증권 레이디스 챔피언십 05.12-14
23141:23266

* 7) 제 11회 E1채리티 오픈 05.26-28
23266:23396

* 8) 롯데 오픈 06.01-04
23396:23514

* 9) 셀트리온 퀸즈 마스터즈 06.09-11
23514:23631
'''

In [139]:
start_loc = 23514
end_loc = 23631

In [140]:
# 예측용 DB
#final = data.iloc[start_loc:end_loc, :]
final = data.iloc[start_loc:, :]

# 정답용 데이터
answer = final.pop('avgStroke')

# 이름판별용
Groupdata.dropna(axis = 0, inplace = True)
Groupdata = Groupdata.sort_index(ascending = True)
#name = Groupdata['name'][start_loc:end_loc] 
name = Groupdata['name'][start_loc:] 

## 6-1) 훈련

In [142]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = data.copy()
y = X.pop('avgStroke')

scaled_data = scaler.fit_transform(X.select_dtypes(exclude=["object", "category"]))
df_scaled = pd.DataFrame(data=scaled_data, columns=X.select_dtypes(exclude=['object', 'category']).columns)
X = df_scaled.copy()


X_train = X.iloc[:start_loc, :] 
y_train = y[:start_loc]

In [143]:
model = CatBoostRegressor(logging_level ='Silent',iterations=750, learning_rate = 0.12, depth = 10)
model.fit(X_train, y_train)

## 6-2) 테스트 데이터 스케일 및 예측

In [144]:
scaled_data = scaler.fit_transform(final.select_dtypes(exclude=["object", "category"]))
df_scaled = pd.DataFrame(data=scaled_data, columns=final.select_dtypes(exclude=['object', 'category']).columns)
X = df_scaled.copy()

In [145]:
y_pred = model.predict(X)

In [146]:
Prediction = pd.DataFrame()
Prediction['event_name'] = final['event_name']
Prediction['name'] = name
Prediction['avgStroke'] = answer
Prediction['pred'] = y_pred

In [147]:
Prediction.to_csv('9_대회결과.csv', encoding = 'cp949', index = False)